In [5]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
from dice_roller import *
from icepool import d

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

In [2]:
# Hit distribution with one or five attacks
from icepool import map, d

def attack_damage(armor_class, attack_roll, attack_modifier):
    if attack_roll == 20:
        return 1.0
    elif attack_roll == 1:
        return 0
    elif attack_roll + attack_modifier >= armor_class:
        return 1.0
    else:
        return 0

# create figure
x_min = -1
x_max = 7
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='hits',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1.0),
        ),
        yaxis=dict(
            title_text='probability',
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


for n in [1,5]:
    n_attacks = n
    atk_mod = 2
    atk_bns = 4
    arm_cls = 12

    name = f'{n} attack' if n == 1 else f'{n} attacks'
    ad = n_attacks @ map(attack_damage, arm_cls, d(20), atk_bns)
    values = np.linspace(x_min, x_max, (x_max - x_min)*4 + 1)
    probabilities = np.array([ad.probability(v) for v in values])
    fig.add_trace(go.Scatter(
        x=np.array(values), 
        y=np.array(probabilities),
        name=f'{name} (mean={ad.mean():.2f}, std={ad.sd():.2f})',
        line_shape='hvh',
        fill='tozeroy',
        hovertemplate=
            f'<b>{name}</b><br>'+
            'hits %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))


# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-simple-hits-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-simple-hits-small')

In [22]:
# plots normalized average damage and damage sigma for an attack that deals fixed damage on a hit or a crit

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='hit + crit probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        yaxis=dict(
            title_text='normalized damage',
            #type='log',
            range=[0, 1],
            #tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
            tickformat='0.1f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def attack_mean(mp, mm, hp, hm, cp, cm):
    return mp*mm + hp*hm + cp*cm

def attack_sigma(mp, mm, ms, hp, hm, hs, cp, cm, cs):
    am = attack_mean(mp, mm, hp, hm, cp, cm)
    return np.sqrt(mp*ms*ms + hp*hs*hs + cp*cs*cs + mp*mm*mm + hp*hm*hm + cp*cm*cm - am*am)

x = np.linspace(0.00, 0.90, 19)

cp = 0.05
dc = 2.0

fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_mean(0.95-x, 0, x, 1, cp, dc),
    mode='lines',
    name=f'average',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'average %{y:.2f}<extra></extra>'
))


fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_sigma(0.95-x, 0, 0, x, 1, 0, cp, dc, 0),
    mode='lines',
    name=f'standard deviation',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'std dev %{y:.2f}<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-attack-normalized-damage-fixed-vs-hit-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-attack-normalized-damage-fixed-vs-hit-small')

In [7]:
# Attack damage CV vs hit probability for constant damage

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='hit + crit probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='CV',
            tickformat='.1f',
            range=[0,3],
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

c = 2.0
p_crit = 1.0/20
p_hit = np.linspace(0,18,19)/20.0
for CV in [0.0]:
    
    fig.add_trace(go.Scatter(
        x=p_crit + p_hit, 
        y=np.sqrt(((CV*CV)*(p_hit + 4*p_crit) + (p_hit + p_crit*c*c - (p_hit + p_crit*c)**2))/((p_hit + p_crit*c)**2)),
        name=f'hit damage CV = {CV:.2f}',
        mode='lines',
        showlegend=False,
        hovertemplate=
            f'hit damage CV {CV:.2f}<br>'+
            'hit + crit probability %{x:.2f}<br>'+
            'attack CV %{y:.2f}<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-fixed-attack-cv-vs-hit-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-fixed-attack-cv-vs-hit-small')

In [26]:
# Attack damage distribution with one or five attacks
from icepool import map, d

def attack_damage(armor_class, attack_roll, attack_bonus, hit_damage_roll, crit_damage_roll):
    if attack_roll == 20:
        return crit_damage_roll
    elif attack_roll == 1:
        return 0
    elif attack_roll + attack_bonus >= armor_class:
        return hit_damage_roll
    else:
        return 0

# create figure
x_min = -1
x_max = 36
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='damage',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)



for n in [1,5]:
    n_attacks = n
    atk_mod = 2
    atk_bns = 4
    arm_cls = 12
    hit_damage = 1 @ d(4) + atk_mod
    crit_damage = 2 @ d(4) + atk_mod

    name = f'{n} attack' if n == 1 else f'{n} attacks'
    ad = n_attacks @ map(attack_damage, arm_cls, d(20), atk_bns, hit_damage, crit_damage)
    values = np.linspace(x_min, x_max, (x_max - x_min) + 1)
    probabilities = np.array([ad.probability(v) for v in values])
    fig.add_trace(go.Scatter(
        x=np.array(values), 
        y=np.array(probabilities),
        name=f'{name} (mean={ad.mean():.2f}, std={ad.sd():.2f})',
        line_shape='hvh',
        fill='tozeroy',
        hovertemplate=
            f'<b>{name}</b><br>'+
            'damage %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))


# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-simple-damage-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-simple-damage-small')

In [68]:
# plots normalized average damage and damage sigma for an attack that deals fixed damage on a hit or a crit

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='hit + crit probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        yaxis=dict(
            title_text='normalized damage',
            #type='log',
            range=[0, 1.2],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            tickformat='0.1f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def attack_mean(mp, mm, hp, hm, cp, cm):
    return mp*mm + hp*hm + cp*cm

def attack_sigma(mp, mm, ms, hp, hm, hs, cp, cm, cs):
    am = attack_mean(mp, mm, hp, hm, cp, cm)
    return np.sqrt(mp*mm*mm + hp*hm*hm + cp*cm*cm - am*am + mp*ms*ms + hp*hs*hs + cp*cs*cs )

x = np.linspace(0.00, 0.90, 19)

cp = 0.05
mc = 2.0
sc = np.sqrt(2.0)
cv = 0.3

fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_mean(0.95-x, 0, x, 1, cp, mc),
    mode='lines',
    name=f'average',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'average %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_sigma(0.95-x, 0, 0, x, 1, 0, cp, mc, 0),
    mode='lines',
    name=f'std dev - attack roll',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'std dev %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_sigma(0.95-x, 0, 0, x, 0, cv, cp, 0, cv*sc),
    mode='lines',
    name=f'std dev - damage roll',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'std dev %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x+cp, 
    y=attack_sigma(0.95-x, 0, 0, x, 1, cv, cp, mc, cv*sc),
    mode='lines',
    name=f'std dev - total',
    hovertemplate=
        'hit + crit %{x:.0%}<br>'+
        'std dev %{y:.2f}<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-attack-normalized-damage-vs-hit-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-attack-normalized-damage-vs-hit-small')

In [65]:
# Attack damage CV vs hit probability

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='hit + crit probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='CV',
            tickformat='.1f',
            range=[0,3],
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

c = 2.0
p_crit = 1.0/20
p_hit = np.linspace(0,18,19)/20.0
for CV in [0.1, 0.2, 0.3]:
    
    fig.add_trace(go.Scatter(
        x=p_crit + p_hit, 
        y=np.sqrt(((CV*CV)*(p_hit + 4*p_crit) + (p_hit + p_crit*c*c - (p_hit + p_crit*c)**2))/((p_hit + p_crit*c)**2)),
        name=f'hit CV = {CV:.2f}',
        mode='lines',
        hovertemplate=
            f'hit CV {CV:.2f}<br>'+
            'hit + crit probability %{x:.2f}<br>'+
            'attack CV %{y:.2f}<extra></extra>'
    ))

"""
# add approximate trendline
fig.add_trace(go.Scatter(
    x=p_crit + p_hit, 
    y=np.sqrt(1/p_hit - 1),
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black')
))
"""

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-attack-cv-vs-hit-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-attack-cv-vs-hit-small')

In [62]:
# Coefficient of variance for a pool of damage split up across multiple attacks
from icepool import map, d

def attack_damage(armor_class, attack_roll, attack_bonus, hit_damage_roll, crit_damage_roll):
    if attack_roll == 20:
        return crit_damage_roll
    elif attack_roll == 1:
        return 0
    elif attack_roll + attack_bonus >= armor_class:
        return hit_damage_roll
    else:
        return 0

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='attacks',
            range=[0,10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='CV',
            range=[0,0.9],
            tickformat='.1f'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

n_dice_pool = 8
d_dice_pool = d(6)
dp = n_dice_pool @ d_dice_pool

n_attacks = [1,2,4,8]

atk_mod = 0
atk_bns = 4
arm_cls = 12

p_hit = 0.6
p_crit = 0.05

# plot CV different number of attacks
mu = []
sigma = []
cv = []
for n in n_attacks:
    n_dice = int(n_dice_pool/n)
    hit_damage = n_dice @ d_dice_pool
    crit_damage = (2*n_dice) @ d_dice_pool

    ad = n @ map(attack_damage, arm_cls, d(20), atk_bns, hit_damage, crit_damage)

    mu += [ad.mean()]
    sigma += [ad.sd()]
    cv += [ad.sd()/ad.mean()]


fig.add_trace(go.Scatter(
    x=np.array(n_attacks), 
    y=np.array(cv),
    showlegend=False,
    hovertemplate=
        'Attacks %{x:.0f}<br>'+
        'CV %{y:.2f}<extra></extra>',
))

# add reference line for minimum CV
m_crit = (2 @ d_dice_pool).mean()/d_dice_pool.mean()
v_crit = (2 @ d_dice_pool).variance()/d_dice_pool.variance()
v_min = 0
v_min += ((p_hit + p_crit*m_crit**2) - (p_hit + p_crit*m_crit)**2)*(n_dice_pool*d_dice_pool.mean()**2)
v_min += (p_hit + p_crit*v_crit)*dp.variance()
s_min = np.sqrt(v_min)

fig.add_trace(go.Scatter(
    x=[0,20], 
    y=2*[s_min/ad.mean()],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

"""
fig.add_trace(go.Scatter(
    x=[0,20], 
    y=2*[np.sqrt((p_hit + p_crit*v_crit)*dp.variance())/ad.mean()],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='red', dash='dash')
))
"""

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-cd-cv-example-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-cd-cv-example-small')

In [ ]:
# approximate attack damage CV vs delta attack bonus

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='delta attack bonus',
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1),
            range=[-5,5],
        ),
        yaxis=dict(
            title_text='CV',
            tickformat='.1f',
            range=[0.4,1.4],
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

c = 2.0
p_crit = 1.0/20
p_hit = np.linspace(0,18,19)/20.0
for CV in [0.2]:
    dp_hit =  p_hit - 0.6

    fig.add_trace(go.Scatter(
        x=20*dp_hit, 
        y=np.sqrt(((CV*CV)*(p_hit + 4*p_crit) + (p_hit + p_crit*c*c - (p_hit + p_crit*c)**2))/((p_hit + p_crit*c)**2)),
        name=f'hit CV = {CV:.2f}',
        mode='lines',
        hovertemplate=
            f'hit CV {CV:.2f}<br>'+
            'hit + crit probability %{x:.2f}<br>'+
            'attack CV %{y:.2f}<extra></extra>'
    ))

    
    fig.add_trace(go.Scatter(
        x=20*dp_hit, 
        #y=np.sqrt((10/7)*((31/70) + CV**2 - 4*(4/10)*((p_hit-0.6)/0.7))),
        #y=np.sqrt(((0.7 + dp_hit)*CV**2 + 0.8 + dp_hit - (0.7 + dp_hit)**2)/((0.7 + dp_hit)**2)),
        #y=np.sqrt((0.31 + 0.7*CV**2 - 0.4*dp_hit + dp_hit*CV**2 - dp_hit**2)/((0.7 + dp_hit)**2)),
        #y=np.sqrt((0.31 + 0.7*CV**2 - 0.4*dp_hit)/((0.7 + dp_hit)**2)),
        #y=(10/7)*np.sqrt((0.31 + 0.7*CV**2 - (9/7)*dp_hit)),
        #y=(10/7)*np.sqrt((0.31 + 0.7*CV**2 - (9/7)*dp_hit + (44/49)*dp_hit**2)),
        y=0.83 + 0.34*(CV - 0.2) - 1.63*dp_hit,
        name=f'hit CV = {CV:.2f} - approx',
        mode='lines',
        line=dict(dash='dash'),
        hovertemplate=
            f'hit CV {CV:.2f}<br>'+
            'hit + crit probability %{x:.2f}<br>'+
            'attack CV %{y:.2f}<extra></extra>'
    ))


# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-attack-cv-vs-hit-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-attack-cv-vs-hit-small')